<a href="https://colab.research.google.com/github/heejinohn/lzho/blob/heejinohn/qrtopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wrds

     |████████████████████████████████| 3.4 MB 27.0 MB/s 


In [2]:
import wrds

In [3]:
db = wrds.Connection()

Enter your WRDS username [root]:hohn
Enter your password:··········
WRDS recommends setting up a .pgpass file.
You can find more info here:
https://www.postgresql.org/docs/9.5/static/libpq-pgpass.html.
Loading library list...
Done


In [5]:
db.describe_table('optionm','opvold')

Approximately 61360236 rows in optionm.opvold.


,name,nullable,type
0,secid,True,DOUBLE_PRECISION
1,date,True,DATE
2,cp_flag,True,VARCHAR(1)
3,volume,True,DOUBLE_PRECISION
4,open_interest,True,DOUBLE_PRECISION
